In [ ]:
# GoogLeNet在名字上致敬LeNet但是已经看不到LeNet的影子，仍然使用串联块的思想

# Inception块 得益于同名电影《盗梦空间》


    与上一节的NIN块相比，Inception块在结构上更加复杂，如图：

In [4]:
# Inception中可以自定义的参数时每个层的输出通道数，以此来控制模型的复杂度

import time
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Inception(nn.Module):
    # c1-c4为每条线路里层的输出通道数
    def __init__(self,in_c,c1,c2,c3,c4):
        super(Inception,self).__init__()
        # 线路1  单1x1卷积层
        self.p1_1 = nn.Conv2d(in_c,c1,kernel_size = 1)
        # 线路2 1x1卷积层后接3x3的卷积层
        self.p2_1 = nn.Conv2d(in_c,c2[0],kernel_size = 1)
        self.p2_2 = nn.Conv2d(in_c,c2[1],kernel_size = 3,padding = 1)
        # 线路3 1x1卷积层接5x5的卷积层
        self.p3_1 = nn.Conv2d(in_c,c3[0],kernel_size = 1)
        self.p3_2 = nn.Conv2d(in_c,c3[1],kernel_size = 5,padding = 1)
        # 线路4 3x3最大值池化层后接1x1的卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size = 3,stride = 1,padding = 1)
        self.p4_2 = nn.Conv2d(in_c,c4,kernel_size = 1)
    
    def forward(self,x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1,p2,p3,p4),dim = 1)# 在通道维上连结输出
        
        

        
# GoogLeNet模型
# GoogLeNet和VGG一样，在主体卷积部分中使用了5个模块，每个模块之间使用步长为
# 2的3x3的最大池化层来减少输出高宽

# 第一个模块使用了一个64通道的7x7的卷积层
b1 = nn.Sequential(
    nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),
    nn.ReLU(),
    nn.MaxUnpool2d(kernel_size=3,stride=2,padding=1),
)
# 第二个模块使用了两个卷积层：首先是64通道的1x1的卷积层
b2 = nn.Sequential(
    nn.Conv2d(64,64,kernel_size=1),
    nn.Conv2d(64,192,kernel_size=3,padding=1),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
)
# 第三个模块串联了2个完整的Inception块
b3 = nn.Sequential(
    Inception(192,64,(96,128),(16,32),32),
    Inception(256,128,(128,192),(32,96),64),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
)
# 第四个模块串联了5个Inception模块
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
# 
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   d2l.GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5, 
                    d2l.FlattenLayer(), nn.Linear(1024, 10))

net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))
X = torch.rand(1, 1, 96, 96)
for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

TypeError: forward() missing 1 required positional argument: 'indices'